# Agent: Tools

이 튜토리얼을 통해 다음을 배울 수 있다:

- LLM의 텍스트 생성 한계와 도구의 필요성을 이해한다.
- OpenAI Function Calling API 사용법을 익힌다.
- 실제 외부 시스템과 연동하는 도구를 구현한다.
- 여러 도구를 조합하여 복잡한 작업을 수행한다.

## 1. 환경 설정

### 1.1 필요한 라이브러리 설치

### 1.2 API 키 설정

In [1]:
from dotenv import load_dotenv
import os

load_dotenv()

True

### 1.3 필요한 라이브러리 가져오기

In [3]:
import json
import requests
from openai import OpenAI

MODEL = "gpt-4o-mini"
client = OpenAI()

## 2. 기본 도구 구현

### 2.1 간단한 도구 함수 만들기
먼저 LLM이 호출할 수 있는 실제 함수를 정의한다.

In [4]:
def get_weather(latitude: float, longitude: float) -> float:
    """주어진 위도와 경도로 현재 날씨 정보를 가져오는 함수다."""
    response = requests.get(
        f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m,wind_speed_10m"
    )
    data = response.json()
    return data["current"]["temperature_2m"]

# 테스트
# 서울의 위도, 경도: 37.5665, 126.9780
seoul_temp = get_weather(37.5665, 126.9780)
print(f"서울의 현재 기온: {seoul_temp}°C")

서울의 현재 기온: 19.5°C


### 2.2 함수 라우터 구현
여러 도구가 있을 때 적절한 함수를 호출하는 라우터를 만든다.

In [5]:
def call_function(name: str, args: dict):
    """함수 이름에 따라 적절한 함수를 호출하는 라우터 함수다."""
    functions = {
        "get_weather": get_weather,
    }
    return functions[name](**args)

## 3. Function Calling 구현

### 3.1 도구 정의 (Tool Schema)
LLM에게 어떤 도구를 사용할 수 있는지 알려주기 위해 도구의 스키마를 정의한다.

In [6]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "제공된 좌표의 현재 온도를 섭씨로 가져온다.",
            "parameters": {
                "type": "object",
                "properties": {
                    "latitude": {
                        "type": "number",
                        "description": "위도 좌표"
                    },
                    "longitude": {
                        "type": "number",
                        "description": "경도 좌표"
                    },
                },
                "required": ["latitude", "longitude"],
            },
        },
    }
]

### 3.2 도구를 사용하는 대화 흐름

In [7]:
def intelligence_with_tools(prompt: str) -> str:
    """도구를 사용하여 지능적으로 응답하는 함수다."""
    
    messages = [{"role": "user", "content": prompt}]
    
    # 1단계: 도구와 함께 모델을 호출한다
    response = client.chat.completions.create(
        model=MODEL,
        messages=messages,
        tools=tools,
        tool_choice="auto",  # 모델이 도구 사용 여부를 자동으로 결정한다
    )
    
    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls
    
    # 2단계: 모델이 도구 호출을 원하는 경우 처리한다
    if tool_calls:
        messages.append(response_message) # LLM의 응답(도구 호출 요청)을 대화 기록에 추가한다
        
        # 3단계: 각 도구 호출을 실행한다
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_args = json.loads(tool_call.function.arguments)
            function_response = call_function(function_name, function_args)
            
            # 4단계: 함수 실행 결과를 메시지에 추가한다
            messages.append({
                "tool_call_id": tool_call.id,
                "role": "tool",
                "name": function_name,
                "content": str(function_response),
            })
        
        # 5단계: 함수 결과를 포함하여 최종 응답을 생성한다
        final_response = client.chat.completions.create(
            model=MODEL,
            messages=messages,
        )
        return final_response.choices[0].message.content
    
    # 도구를 사용하지 않은 경우, 첫 번째 응답을 바로 반환한다
    return response_message.content

# 사용 예시
result = intelligence_with_tools("오늘 서울 날씨는 어떤가?")
print(result)

오늘 서울의 기온은 약 19.4도입니다. 날씨는 맑거나 구름이 약간 낀 상태일 가능성이 높습니다. 외출 시 가벼운 옷차림이 적합할 것 같습니다.


## 4. 실전 예제

### 4.1 계산기 도구
복잡한 수학 연산을 수행하는 도구다.

In [8]:
def calculate(expression: str) -> float:
    """수학 표현식을 계산하는 함수다."""
    # 주의: eval()은 보안에 취약할 수 있으므로 실제 프로덕션에서는 안전한 파서를 사용해야 한다.
    return eval(expression)

# 도구 정의
calculator_tool = {
    "type": "function",
    "function": {
        "name": "calculate",
        "description": "수학 표현식을 계산한다. 예: '2 + 2', '15 * 7', '100 / 4'",
        "parameters": {
            "type": "object",
            "properties": {
                "expression": {
                    "type": "string",
                    "description": "계산할 수학 표현식"
                },
            },
            "required": ["expression"],
        },
    },
}

# 사용 예시
def ask_with_calculator(question: str) -> str:
    tools = [calculator_tool]
    messages = [{"role": "user", "content": question}]
    
    response = client.chat.completions.create(
        model=MODEL,
        messages=messages,
        tools=tools,
        tool_choice="auto",
    )
    
    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls
    
    if tool_calls:
        messages.append(response_message)
        
        for tool_call in tool_calls:
            function_args = json.loads(tool_call.function.arguments)
            result = calculate(function_args["expression"])
            
            messages.append({
                "tool_call_id": tool_call.id,
                "role": "tool",
                "name": "calculate",
                "content": str(result),
            })
        
        final_response = client.chat.completions.create(
            model=MODEL,
            messages=messages,
        )
        return final_response.choices[0].message.content
    
    return response_message.content

print(ask_with_calculator("847293 곱하기 652847은 얼마야?"))
# LLM이 calculate 도구를 사용하여 정확한 답을 계산한다.

847293 곱하기 652847은 553,152,693,171입니다.


### 4.2 데이터베이스 조회 도구
사용자 정보를 조회하는 가상의 데이터베이스 도구다.

In [9]:
# 가상 데이터베이스
users_db = {
    "user_001": {"name": "김철수", "age": 28, "city": "서울"},
    "user_002": {"name": "이영희", "age": 34, "city": "부산"},
    "user_003": {"name": "박민수", "age": 45, "city": "대구"},
}

def get_user_info(user_id: str) -> dict:
    """사용자 ID로 사용자 정보를 조회하는 함수다."""
    return users_db.get(user_id, {"error": "사용자를 찾을 수 없습니다"})

# 도구 정의
user_info_tool = {
    "type": "function",
    "function": {
        "name": "get_user_info",
        "description": "사용자 ID로 사용자의 이름, 나이, 도시 정보를 조회한다.",
        "parameters": {
            "type": "object",
            "properties": {
                "user_id": {
                    "type": "string",
                    "description": "조회할 사용자의 ID (예: user_001)"
                },
            },
            "required": ["user_id"],
        },
    },
}

# 사용 예시
def query_user(question: str) -> str:
    tools = [user_info_tool]
    messages = [{"role": "user", "content": question}]
    
    response = client.chat.completions.create(
        model=MODEL,
        messages=messages,
        tools=tools,
    )
    
    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls
    
    if tool_calls:
        messages.append(response_message)
        
        for tool_call in tool_calls:
            function_args = json.loads(tool_call.function.arguments)
            result = get_user_info(function_args["user_id"])
            
            messages.append({
                "tool_call_id": tool_call.id,
                "role": "tool",
                "name": "get_user_info",
                "content": json.dumps(result, ensure_ascii=False),
            })
        
        final_response = client.chat.completions.create(
            model=MODEL,
            messages=messages,
        )
        return final_response.choices[0].message.content
    
    return response_message.content

print(query_user("user_001의 정보를 알려줘"))

user_001의 정보는 다음과 같습니다:

- 이름: 김철수
- 나이: 28세
- 도시: 서울


### 4.3 파일 작업 도구
텍스트 파일을 읽고 쓰는 도구다.

In [10]:
def read_file(filename: str) -> str:
    """파일 내용을 읽어오는 함수다."""
    try:
        with open(filename, 'r', encoding='utf-8') as f:
            return f.read()
    except FileNotFoundError:
        return f"오류: 파일 '{filename}'을 찾을 수 없다."
    except Exception as e:
        return f"파일 읽기 중 오류 발생: {e}"

def write_file(filename: str, content: str) -> str:
    """파일에 내용을 쓰는 함수다."""
    try:
        with open(filename, 'w', encoding='utf-8') as f:
            f.write(content)
        return f"파일 '{filename}'에 저장 완료"
    except Exception as e:
        return f"파일 쓰기 중 오류 발생: {e}"

# 도구 정의
file_tools = [
    {
        "type": "function",
        "function": {
            "name": "read_file",
            "description": "지정된 파일의 내용을 읽어온다.",
            "parameters": {
                "type": "object",
                "properties": {
                    "filename": {"type": "string", "description": "읽을 파일 이름"},
                },
                "required": ["filename"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "write_file",
            "description": "지정된 파일에 내용을 저장한다.",
            "parameters": {
                "type": "object",
                "properties": {
                    "filename": {"type": "string", "description": "저장할 파일 이름"},
                    "content": {"type": "string", "description": "저장할 내용"},
                },
                "required": ["filename", "content"],
            },
        },
    },
]

# 파일 함수 라우터
def call_file_function(name: str, args: dict):
    """파일 관련 함수를 호출하는 라우터다."""
    functions = {
        "read_file": read_file,
        "write_file": write_file,
    }
    return functions[name](**args)

## 5. 여러 도구를 조합하기

### 5.1 Pydantic을 활용한 타입 안전한 도구 정의
Pydantic을 사용하면 도구의 파라미터를 타입 안전하게 정의하고 자동으로 검증할 수 있다.

#### 5.1.1 설치 및 기본 사용법

In [11]:
from pydantic import BaseModel, Field
from typing import Optional

class WeatherParams(BaseModel):
    """날씨 조회 파라미터 모델이다."""
    latitude: float = Field(..., description="위도 좌표")
    longitude: float = Field(..., description="경도 좌표")

class CalculateParams(BaseModel):
    """계산기 파라미터 모델이다."""
    expression: str = Field(..., description="계산할 수학 표현식")

# Pydantic 모델을 JSON Schema로 변환
def pydantic_to_function_schema(name: str, description: str, model: type[BaseModel]) -> dict:
    """Pydantic 모델을 OpenAI Function Schema로 변환한다."""
    return {
        "type": "function",
        "function": {
            "name": name,
            "description": description,
            "parameters": model.model_json_schema(),
        },
    }

# 도구 정의
weather_tool_pydantic = pydantic_to_function_schema(
    name="get_weather",
    description="제공된 좌표의 현재 온도를 섭씨로 가져온다.",
    model=WeatherParams
)

calculator_tool_pydantic = pydantic_to_function_schema(
    name="calculate",
    description="수학 표현식을 계산한다.",
    model=CalculateParams
)

print(json.dumps(weather_tool_pydantic, indent=2, ensure_ascii=False))

{
  "type": "function",
  "function": {
    "name": "get_weather",
    "description": "제공된 좌표의 현재 온도를 섭씨로 가져온다.",
    "parameters": {
      "description": "날씨 조회 파라미터 모델이다.",
      "properties": {
        "latitude": {
          "description": "위도 좌표",
          "title": "Latitude",
          "type": "number"
        },
        "longitude": {
          "description": "경도 좌표",
          "title": "Longitude",
          "type": "number"
        }
      },
      "required": [
        "latitude",
        "longitude"
      ],
      "title": "WeatherParams",
      "type": "object"
    }
  }
}


#### 5.1.2 복잡한 파라미터 모델

In [12]:
from pydantic import BaseModel, Field
from typing import Optional, Literal
from datetime import datetime

class SearchProductParams(BaseModel):
    """제품 검색 파라미터 모델이다."""
    query: str = Field(..., description="검색할 제품 이름 또는 키워드")
    category: Optional[str] = Field(None, description="제품 카테고리 (전자제품, 의류, 식품 등)")
    min_price: Optional[float] = Field(None, ge=0, description="최소 가격")
    max_price: Optional[float] = Field(None, ge=0, description="최대 가격")
    in_stock: bool = Field(True, description="재고 있는 제품만 검색")
    sort_by: Literal["price", "rating", "newest"] = Field("rating", description="정렬 기준")

class UserInfoParams(BaseModel):
    """사용자 정보 조회 파라미터 모델이다."""
    user_id: str = Field(..., description="사용자 ID", min_length=3)
    include_orders: bool = Field(False, description="주문 내역 포함 여부")

# 함수 구현
def search_products(query: str, category: Optional[str] = None, 
                   min_price: Optional[float] = None, max_price: Optional[float] = None,
                   in_stock: bool = True, sort_by: str = "rating") -> list:
    """제품을 검색하는 함수다."""
    print(f"Searching for: {query}, Category: {category}, Price: {min_price}-{max_price}")
    products = [
        {"name": "무선 마우스", "price": 25000, "category": "전자제품", "rating": 4.5},
        {"name": "키보드", "price": 89000, "category": "전자제품", "rating": 4.8},
    ]
    return [p for p in products if query.lower() in p["name"].lower()]

#### 5.1.3 타입 검증이 포함된 통합 에이전트

In [13]:
from pydantic import BaseModel, ValidationError
import json

class PydanticToolAgent:
    """Pydantic 모델을 사용하는 타입 안전한 에이전트다."""
    
    def __init__(self):
        self.client = OpenAI()
        self.tools = []
        self.functions = {}
        self.param_models = {}
    
    def add_tool(self, name: str, description: str, 
                 function: callable, param_model: type[BaseModel]):
        """Pydantic 모델과 함께 도구를 추가한다."""
        tool_schema = pydantic_to_function_schema(name, description, param_model)
        self.tools.append(tool_schema)
        self.functions[name] = function
        self.param_models[name] = param_model
    
    def execute_function(self, name: str, args: dict):
        """파라미터를 검증한 후 함수를 실행한다."""
        try:
            param_model = self.param_models[name]
            validated_params = param_model(**args)
            return self.functions[name](**validated_params.model_dump())
        except ValidationError as e:
            return f"파라미터 오류: {e}"
        except Exception as e:
            return f"함수 실행 오류: {e}"
    
    def chat(self, message: str) -> str:
        """사용자 메시지를 처리한다."""
        messages = [{"role": "user", "content": message}]
        
        response = self.client.chat.completions.create(
            model=MODEL,
            messages=messages,
            tools=self.tools,
        )
        
        response_message = response.choices[0].message
        tool_calls = response_message.tool_calls
        
        if tool_calls:
            messages.append(response_message)
            
            for tool_call in tool_calls:
                function_name = tool_call.function.name
                function_args = json.loads(tool_call.function.arguments)
                result = self.execute_function(function_name, function_args)
                
                messages.append({
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": str(result),
                })
            
            final_response = self.client.chat.completions.create(
                model=MODEL,
                messages=messages,
            )
            return final_response.choices[0].message.content
        
        return response_message.content

# 사용 예시
pydantic_agent = PydanticToolAgent()

pydantic_agent.add_tool(
    name="search_products",
    description="제품을 검색한다.",
    function=search_products,
    param_model=SearchProductParams
)

print(pydantic_agent.chat("5만원 이하의 무선 마우스를 찾아줘"))

Searching for: 무선 마우스, Category: None, Price: None-50000.0
5만원 이하의 무선 마우스를 찾았습니다. 다음은 제품 정보입니다:

- **제품명**: 무선 마우스
- **가격**: 25,000원
- **카테고리**: 전자제품
- **평점**: 4.5

필요한 정보가 더 있으면 말씀해 주세요!


### 5.2 통합 에이전트 클래스
여러 도구를 관리하는 에이전트 클래스를 만든다.

In [14]:
class ToolAgent:
    """여러 도구를 사용할 수 있는 AI 에이전트 클래스다."""
    
    def __init__(self):
        self.client = OpenAI()
        self.tools = []
        self.functions = {}
    
    def add_tool(self, tool_schema: dict, function: callable):
        """도구를 에이전트에 추가한다."""
        self.tools.append(tool_schema)
        function_name = tool_schema["function"]["name"]
        self.functions[function_name] = function
    
    def execute_function(self, name: str, args: dict):
        """등록된 함수를 실행한다."""
        return self.functions[name](**args)
    
    def chat(self, message: str) -> str:
        """사용자 메시지를 처리하고 필요시 도구를 사용한다."""
        messages = [{"role": "user", "content": message}]
        
        response = self.client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
            tools=self.tools,
        )
        
        response_message = response.choices[0].message
        tool_calls = response_message.tool_calls
        
        if tool_calls:
            messages.append(response_message)
            
            for tool_call in tool_calls:
                function_name = tool_call.function.name
                function_args = json.loads(tool_call.function.arguments)
                result = self.execute_function(function_name, function_args)
                
                messages.append({
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": str(result),
                })
            
            final_response = self.client.chat.completions.create(
                model=MODEL,
                messages=messages,
            )
            return final_response.choices[0].message.content
        
        return response_message.content

# 사용 예시
agent = ToolAgent()

# 날씨 도구 추가
agent.add_tool(tools[0], get_weather) # 3.1에서 정의한 weather tool

# 계산기 도구 추가
agent.add_tool(calculator_tool, calculate)

# 테스트
print(agent.chat("서울(위도 37.5665, 경도 126.9780)의 현재 기온은?"))
print(agent.chat("123 곱하기 456은?"))

서울의 현재 기온은 19.4도입니다.
123 곱하기 456은 56,088입니다.


### 5.3 복합 작업 수행
여러 도구를 순차적으로 사용하는 예제다.

In [15]:
def get_currency_rate(from_currency: str, to_currency: str) -> float:
    """환율 정보를 가져오는 가상 함수다."""
    rates = {
        ("USD", "KRW"): 1350.0,
        ("EUR", "KRW"): 1450.0,
        ("JPY", "KRW"): 9.0, # 100엔 기준이 아닌 1엔 기준
    }
    return rates.get((from_currency, to_currency), 1.0)

# 환율 도구 정의
currency_tool = {
    "type": "function",
    "function": {
        "name": "get_currency_rate",
        "description": "두 통화 간의 환율을 조회한다.",
        "parameters": {
            "type": "object",
            "properties": {
                "from_currency": {"type": "string", "description": "원본 통화 코드(예: USD)"},
                "to_currency": {"type": "string", "description": "대상 통화 코드(예: KRW)"},
            },
            "required": ["from_currency", "to_currency"],
        },
    },
}

# 새로운 에이전트를 만들고 도구들을 추가한다
financial_agent = ToolAgent()
financial_agent.add_tool(currency_tool, get_currency_rate)
financial_agent.add_tool(calculator_tool, calculate)

# 복합 질문
print(financial_agent.chat("150 유로는 한국 원화로 얼마야? 계산해서 정확한 금액을 알려줘"))

현재 환율에 따라 150 유로는 약 217,500 원입니다. (1 유로 = 1,450 원 기준)


## 6. 도구 사용 모범 사례

### 6.1 도구 선택 제어
`tool_choice` 매개변수로 도구 사용을 제어할 수 있다.

- `"auto"` (기본값): LLM이 필요하다고 판단할 때 자동으로 도구를 선택한다.
- `"required"`: 반드시 등록된 도구 중 하나를 사용하도록 강제한다.
- `{"type": "function", "function": {"name": "get_weather"}}`: 특정 도구의 사용을 강제한다.
- `"none"`: 도구를 절대 사용하지 않도록 강제한다. LLM은 도구 없이 텍스트로만 답변한다.

### 6.2 결과 형식 표준화
도구의 반환 값을 일관된 형식(예: JSON)으로 유지하면 LLM이 결과를 더 잘 이해하고 처리할 수 있다.

In [16]:
def get_product_info(product_id: str) -> dict:
    """제품 정보를 표준 형식으로 반환한다."""
    products = {
        "prod_001": {
            "name": "무선 마우스",
            "price": 25000,
            "stock": 15,
            "category": "전자제품"
        }
    }
    
    product = products.get(product_id)
    
    return {
        "success": True if product else False,
        "data": product,
        "message": "조회 완료" if product else "제품을 찾을 수 없습니다"
    }